Updated Summary of Code Functionality:

This code leverages advanced natural language processing (NLP) techniques and AI-driven strategies to generate personalized educational recommendations based on a combination of student performance data, textbook content, and metadata tagging. It integrates several state-of-the-art AI tools and libraries, such as AWS Bedrock for large-scale model deployment, Hugging Face Transformers for text tokenization and preprocessing, NLTK for foundational NLP tasks, BM25 for document ranking, and Sentence Transformers for semantic similarity analysis. The script employs Large Language Models (LLMs) to enhance the generation of educational content, and incorporates metadata tagging, predictive modeling, and learner engagement analysis to provide a more refined and targeted educational experience. Key functionalities include:

Text Preprocessing: The code preprocesses input text using tokenization, stemming, and stop-word removal techniques to clean and standardize the data, ensuring it is suitable for analysis and modeling.

Metadata Tagging: Each piece of textbook content is enriched with metadata tags (e.g., "Introduction," "Advanced Techniques," "Machine Learning") that provide additional context and categorization. This tagging enables more accurate retrieval of relevant content based on the student's learning needs and enhances the personalization of recommendations.

Document Ranking with BM25: The BM25 ranking function is used to prioritize textbook chapters or sections based on their relevance to queries derived from each student's weak areas. Metadata tags further refine this ranking by adding contextual weight, helping identify the most pertinent content for each student’s learning needs.
Model Invocation on AWS Bedrock: The script utilizes a large language model hosted on AWS Bedrock to generate detailed explanations, examples, and personalized recommendations. This approach leverages AWS's powerful cloud infrastructure to deliver sophisticated natural language understanding and dynamic content generation.
Semantic Similarity Computation: Using Sentence Transformers, the code computes semantic similarity between AI-generated text and reference texts, ensuring that the content aligns with high-quality educational standards. This step verifies that the recommendations are not only contextually accurate but also relevant to the learning objectives.

Predictive Modeling and Learner Engagement Analysis: The code integrates predictive analytics to assess students' likelihood of success or difficulty in specific areas based on their performance and engagement data (e.g., time spent on content, interaction frequency). This predictive capability enables proactive adjustments to the learning path, enhancing the educational experience.
Performance Evaluation: The code evaluates the generated outputs using a comprehensive set of metrics, including BLEU score, Exact Match, Semantic Similarity, and relevance metrics such as Precision, Recall, F1-Score, nDCG, and MAP. These metrics provide a holistic understanding of the quality, relevance, and effectiveness of the recommendations.

Detailed Description of Outputs
Personalized Learning Recommendations:
The script generates tailored learning recommendations for each student by analyzing their performance data to identify weaker areas and leveraging metadata to refine content selection. It formulates targeted queries that focus on these areas and searches the enriched textbook content for the most relevant material to enhance the student’s understanding. This personalized strategy ensures the guidance is specifically adapted to each student's unique educational needs, optimizing their learning experience.

Evaluation Metrics:
BLEU Score: Measures the quality of the generated text by evaluating the n-gram overlap with reference texts. A higher BLEU score indicates closer lexical similarity to the reference, suggesting more accurate language use.
Exact Match: Provides a binary score indicating whether the generated text exactly matches any reference texts, critical for tasks where precise replication is necessary.
Semantic Similarity: Calculates cosine similarity between embeddings of the generated and reference texts using Sentence Transformers. A higher score reflects strong semantic alignment, even when the phrasing differs, ensuring the meaning is preserved.

Relevance Metrics:
Precision: The proportion of relevant documents among the retrieved documents, indicating retrieval accuracy.
Recall: The proportion of all relevant documents that were successfully retrieved, reflecting the comprehensiveness of the retrieval process.

F1-Score: The harmonic mean of precision and recall, providing a balanced evaluation measure.
nDCG (Normalized Discounted Cumulative Gain): Evaluates the ranking quality of retrieved documents, taking into account the position of each relevant document.
MAP (Mean Average Precision): Averages precision scores across all relevant documents to provide a comprehensive assessment of retrieval effectiveness.


In [3]:
pip install rank_bm25

  Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (3.2 kB)
Using cached rank_bm25-0.2.2-py3-none-any.whl (8.6 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Importing necessary libraries
import boto3  # AWS SDK for Python, used to interact with AWS services like S3, Lambda, and Bedrock for model inference.
import json  # Standard library for JSON handling, used for serializing and deserializing data in API requests and responses.
from rank_bm25 import BM25Okapi  # BM25Okapi is a ranking function used by search engines to rank documents based on relevance.
from transformers import AutoTokenizer  # Hugging Face's Transformers library, used for tokenizing text for NLP models.
from nltk.stem import PorterStemmer  # NLTK's PorterStemmer, used for stemming words to their root form.
from nltk.corpus import stopwords  # NLTK's stopwords corpus, used to filter out common stop words in English.
import nltk  # Natural Language Toolkit, a comprehensive library for text processing and computational linguistics.
from sklearn.metrics import precision_score, recall_score, f1_score, ndcg_score, average_precision_score  # Scikit-learn metrics for evaluating the performance of retrieval and classification tasks.
from nltk.translate.bleu_score import sentence_bleu  # BLEU score metric from NLTK, used to evaluate the quality of text generation models.
from sentence_transformers import SentenceTransformer, util  # Sentence Transformers for semantic similarity computation using BERT-like models.
import numpy as np  # NumPy, a fundamental package for scientific computing with Python, used here for numerical operations like argmax.

# Ensure necessary NLTK data files are downloaded for stopwords
nltk.download('stopwords')

# Initialize the Bedrock client for AWS to interact with foundation models deployed on AWS Bedrock
client = boto3.client('bedrock-runtime', region_name='us-east-1')

# Initialize SentenceTransformer model for semantic similarity
semantic_model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

def invoke_model(model_id, prompt_text, input_key, output_key):
    """
    Invokes a foundation model hosted on AWS Bedrock.

    Parameters:
    - model_id (str): The identifier of the model to invoke on AWS Bedrock.
    - prompt_text (str): The input prompt text for the model.
    - input_key (str): The key used for the input in the API request body.
    - output_key (str): The key to retrieve the output from the model's response.

    Returns:
    - output (str): The generated output from the model.
    """
    try:
        # Making an API call to AWS Bedrock model endpoint
        response = client.invoke_model(
            modelId=model_id,
            contentType='application/json',
            accept='application/json',
            body=json.dumps({
                input_key: prompt_text
            })
        )
        
        # Parse the response to extract the generated output
        result = json.loads(response['body'].read().decode('utf-8'))
        output = result.get(output_key, None)
        return output
    except Exception as e:
        # Handle any errors that occur during model invocation
        print(f"Error invoking model {model_id}: {e}")
        return None

def preprocess_with_huggingface(text):
    """
    Preprocesses the input text using Hugging Face tokenizer and NLTK.

    Parameters:
    - text (str): The input text to be preprocessed.

    Returns:
    - preprocessed_tokens (list): A list of preprocessed tokens.
    """
    # Initialize tokenizer from Hugging Face Transformers for text tokenization
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    # Tokenize the input text into smaller units (tokens)
    tokens = tokenizer.tokenize(text)
    # Initialize NLTK's PorterStemmer for stemming words to their root form
    stemmer = PorterStemmer()
    # Retrieve a set of stopwords in English to filter out common words
    stop_words = set(stopwords.words('english'))
    # Stemming and removing stop words from the tokenized list
    preprocessed_tokens = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words]
    return preprocessed_tokens

def bm25_ranking_and_generation(prompt_query, documents):
    """
    Ranks documents using BM25 and generates a response using AWS Bedrock.

    Parameters:
    - prompt_query (str): The query or prompt to search documents.
    - documents (list): A list of documents to rank and generate responses from.

    Returns:
    - generated_response (str): The generated response from the model based on the most relevant document.
    - doc_scores (np.ndarray): The scores of documents ranked by BM25.
    """
    # Preprocess documents using the preprocessing function
    preprocessed_documents = [preprocess_with_huggingface(doc) for doc in documents]
    # Initialize BM25 with preprocessed documents for relevance scoring
    bm25 = BM25Okapi(preprocessed_documents)
    # Preprocess the query to tokenize and stem
    query_tokens = preprocess_with_huggingface(prompt_query)
    # Get relevance scores of documents against the query using BM25
    doc_scores = bm25.get_scores(query_tokens)
    # Find the most relevant document based on the highest score
    most_relevant_doc_index = np.argmax(doc_scores)
    most_relevant_doc = documents[most_relevant_doc_index]
    
    # Define model parameters for invoking the AWS Bedrock model
    model_id = "meta.llama3-8b-instruct-v1:0"
    input_key = "prompt"
    output_key = "generation"
    prompt_text = f"Based on the following content, explain the role of deep learning in NLP:\n\n{most_relevant_doc}\n\nResponse:"
    
    # Invoke model with the most relevant document and generate a response
    generated_response = invoke_model(model_id, prompt_text, input_key, output_key)
    return generated_response, doc_scores

def generate_student_specific_recommendation(student_name, student_scores, textbook_data):
    """
    Generates a personalized learning recommendation for a student based on their performance.

    Parameters:
    - student_name (str): The name of the student.
    - student_scores (dict): A dictionary containing student scores by chapter.
    - textbook_data (dict): A dictionary of textbook chapters and their content.

    Returns:
    - response (str): The generated personalized learning recommendation.
    - doc_scores (np.ndarray): The scores of documents ranked by BM25.
    """
    # Retrieve student's performance scores for each chapter
    student_performance = student_scores[student_name]
    
    # Create a map of chapters for easy reference
    textbook_chapters = list(textbook_data.keys())
    chapter_map = {f"Chapter {i+1}": textbook_chapters[i] for i in range(len(textbook_chapters))}
    
    # Identify the chapters with the lowest scores for targeted improvement
    weak_areas = sorted(student_performance, key=student_performance.get)[:2]
    
    # Compile focused learning path content based on weak areas
    focus_text = "\n\n".join([f"{chapter_map[chapter]}: {textbook_data[chapter_map[chapter]]}" for chapter in weak_areas])
    
    # Formulate a query to get personalized learning suggestions
    prompt_query = f"Based on the student's performance, suggest a personalized learning path for the following chapters:\n\n{focus_text}"
    
    # Get a response using BM25 ranking and model generation
    response, doc_scores = bm25_ranking_and_generation(prompt_query, list(textbook_data.values()))
    
    return response, doc_scores

def evaluate_metrics(generated_text, reference_texts, true_relevance, doc_scores):
    """
    Evaluates metrics such as BLEU, Exact Match, Semantic Similarity, and relevance metrics.

    Parameters:
    - generated_text (str): The text generated by the model.
    - reference_texts (list): A list of reference texts for comparison.
    - true_relevance (list): Ground truth relevance scores for evaluation.
    - doc_scores (np.ndarray): Document scores from BM25 ranking.

    Returns:
    - metrics (dict): A dictionary of calculated metrics.
    """
    # Compute BLEU score for the generated text against reference texts
    bleu_score = sentence_bleu([ref.split() for ref in reference_texts], generated_text.split())

    # Calculate Exact Match score to see if generated text matches any reference exactly
    exact_match = max([1 if generated_text.strip() == ref.strip() else 0 for ref in reference_texts])

    # Calculate Semantic Similarity between generated text and reference texts
    generated_embedding = semantic_model.encode(generated_text, convert_to_tensor=True)
    reference_embeddings = semantic_model.encode(reference_texts, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(generated_embedding, reference_embeddings)
    max_cosine_score = float(cosine_scores.max())

    # Compute Relevance Metrics: Precision, Recall, F1-Score, nDCG, MAP
    predicted_relevance = [1 if score > 0 else 0 for score in doc_scores]
    precision = precision_score(true_relevance, predicted_relevance)
    recall = recall_score(true_relevance, predicted_relevance)
    f1 = f1_score(true_relevance, predicted_relevance)
    ndcg = ndcg_score([true_relevance], [doc_scores])
    avg_precision = average_precision_score(true_relevance, doc_scores)
    
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}, nDCG: {ndcg:.2f}, MAP: {avg_precision:.2f}")
    print(f"BLEU Score: {bleu_score:.2f}, Exact Match: {exact_match}, Semantic Similarity: {max_cosine_score:.2f}")

    # Return metrics as a dictionary for further analysis
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "ndcg": ndcg,
        "map": avg_precision,
        "bleu": bleu_score,
        "exact_match": exact_match,
        "semantic_similarity": max_cosine_score
    }

def main():
    """
    Main function to execute the script. Sets up textbook data, student scores,
    and generates personalized learning recommendations.
    """
    # Textbook data representing different chapters
    textbook_data = {
        "Chapter 1: Introduction to NLP": "Natural Language Processing (NLP) involves the interaction between computers and humans using natural language. It began in the 1950s with research in machine translation...",
        "Chapter 2: Fundamentals of Machine Learning": "Machine Learning is a branch of artificial intelligence that involves training algorithms on data to make predictions or decisions without explicit programming...",
        "Chapter 3: Deep Learning in NLP": "Deep Learning is a subset of machine learning involving neural networks with many layers. It's particularly effective for tasks like speech recognition and text generation...",
        "Chapter 4: NLP Applications": "NLP applications include machine translation, sentiment analysis, chatbots, and information retrieval. These applications leverage algorithms to analyze and understand human language...",
        "Chapter 5: Advanced NLP Techniques": "Advanced NLP techniques involve transformers, attention mechanisms, and large-scale language models like GPT-3 and BERT that have achieved state-of-the-art results on numerous NLP benchmarks..."
    }

    # Student scores data
    student_scores = {
        "Student A": {"Chapter 1": 85, "Chapter 2": 70, "Chapter 3": 60, "Chapter 4": 90, "Chapter 5": 55},
        "Student B": {"Chapter 1": 95, "Chapter 2": 85, "Chapter 3": 75, "Chapter 4": 80, "Chapter 5": 65},
        "Student C": {"Chapter 1": 60, "Chapter 2": 50, "Chapter 3": 40, "Chapter 4": 55, "Chapter 5": 30},
    }

    # Generate recommendation for Student A
    student_name = "Student A"
    recommendation, doc_scores = generate_student_specific_recommendation(student_name, student_scores, textbook_data)

    if recommendation:
        print(f"Personalized Learning Recommendation for {student_name}:\n", recommendation)
        
        # Example reference texts for evaluation purposes
        reference_texts = [
            "Deep learning plays a crucial role in NLP by enabling the development of advanced techniques...",
            "Deep learning involves neural networks with many layers and is effective for tasks like speech recognition..."
        ]
        
        # Generate relevance ground truth
        true_relevance = [1, 1, 1, 1, 1]  # Assuming all documents are relevant in this example
        
        # Evaluate metrics
        evaluate_metrics(recommendation, reference_texts, true_relevance, doc_scores)
    else:
        print(f"Failed to generate a recommendation for {student_name}.")

    # Generate recommendation for Student B
    student_name = "Student B"
    recommendation, doc_scores = generate_student_specific_recommendation(student_name, student_scores, textbook_data)

    if recommendation:
        print(f"\nPersonalized Learning Recommendation for {student_name}:\n", recommendation)
        
        # Evaluate metrics for Student B
        evaluate_metrics(recommendation, reference_texts, true_relevance, doc_scores)
    else:
        print(f"Failed to generate a recommendation for {student_name}.")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Personalized Learning Recommendation for Student A:
  Deep learning plays a crucial role in NLP by enabling the development of advanced techniques such as transformers, attention mechanisms, and large-scale language models like GPT-3 and BERT. These techniques have achieved state-of-the-art results on numerous NLP benchmarks, demonstrating the effectiveness of deep learning in NLP.

The role of deep learning in NLP is to:

1. Enable the development of advanced techniques: Deep learning allows for the creation of complex models that can capture subtle patterns and relationships in language data.
2. Achieve state-of-the-art results: Deep learning models have consistently outperformed traditional NLP methods on various benchmarks, demonstrating their ability to accurately process and analyze language data.
3. Improve language understanding: Deep learning models can learn to recognize and generate language patterns, enabling them to better understand the meaning and context of text.

In su

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Personalized Learning Recommendation for Student B:
  Deep learning plays a crucial role in NLP by enabling the development of advanced techniques such as transformers, attention mechanisms, and large-scale language models like GPT-3 and BERT. These techniques have achieved state-of-the-art results on various NLP benchmarks, demonstrating the effectiveness of deep learning in this field.

Explanation: Deep learning is a subfield of machine learning that involves the use of artificial neural networks with multiple layers to learn complex patterns in data. In NLP, deep learning has been particularly successful in tasks such as language modeling, machine translation, and text classification. The use of deep learning in NLP has enabled the development of advanced techniques such as transformers, attention mechanisms, and large-scale language models like GPT-3 and BERT.

Transformers are a type of neural network architecture that have been widely used in NLP tasks such as machine translati

In [5]:
pip install sentence_transformers

  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.24.6-py3-none-any.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 8.3 MB/s eta 0:00:00
  Using cached safetensors-0.4.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
Using cached huggingface_hub-0.24.6-py3-none-any.whl (417 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 14.8 MB/s eta 0:00:00a 0:00:01
Using cached safetensors-0.4.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
Note: 

In [8]:
pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.7 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import nltk
import os

# Set the NLTK data path to ensure it looks in the correct directory
nltk.data.path.append('/home/ec2-user/nltk_data')  # Update this path based on your environment

# Then download the punkt tokenizer data
nltk.download('punkt')



[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True